# GW AGN Example

In [6]:
import numpy as np
from astropy.table import Table, vstack, hstack
from astropy.coordinates import SkyCoord
import astropy.units as u
import sys, os
sys.path.append('/mnt/rdliang/AGN/BBH_AGN/Cmash')
import Cmash
from astropy.time import Time

## O4a: Cross-match EP Known Source with AGN flare detected in Optical

In [5]:
agn_flare_table = Table.read('/mnt/rdliang/AGN/BBH_AGN/data/AGNflares.csv')
agn_flare_cat = SkyCoord(agn_flare_table['RA'],agn_flare_table['DEC'],unit=u.deg)

EP_source_table = Table.read('/mnt/rdliang/AGN/BBH_AGN/data/stp_source_list.csv')
EP_source_cat = SkyCoord(EP_source_table['RA'],EP_source_table['Dec'],unit=u.deg)

source_matched_idx, cat_matched_idx = Cmash.match_cat(agn_flare_cat,EP_source_cat,radius=3*u.arcmin)

agn_flare_matched, EP_source_matched = agn_flare_table[source_matched_idx], EP_source_table[cat_matched_idx]
matched_table = hstack((agn_flare_matched,EP_source_matched))
matched_table.write('/mnt/rdliang/AGN/BBH_AGN/data/AGNflare_EP_matched.csv',format='csv')

In [11]:
Time('2024-06-01 12:00:00',scale='utc').mjd

60462.5

In [8]:
Time(60224.6,format='mjd').iso

'2023-10-07 14:24:00.000'